In [ ]:
import requests
from datetime import datetime
import time
from lxml import html
from fake_useragent import UserAgent

import pymongo
client = pymongo.MongoClient()
db = client.regulationsgov
documents_collection = db.documents
comments_collection = db.comments

from RegAPI import Reg_API

In [ ]:
page_size = 250
api = Reg_API(page_size)

In [ ]:
def scrapeAll(endpoint, collection):
    pageNum = 1
    while True: 
        documents = api.endpoint(endpoint).sort("lastModifiedDate").page(pageNum).get()
        for doc in documents["data"]:
            doc["_id"] = doc["id"]
            doc["_inserted_time"] = time.time()
        
        collection.insert_many(documents["data"])
        
        if pageNum > 1: ### Comment out/delete to run for ALL pages
            break       # - Currently stops after 2nd page
        
        if documents["meta"]["hasNextPage"] == False:
            date = documents["data"][-1]["attributes"]["lastModifiedDate"]
        else:
            pageNum += 1

scrapeAll("/comments", comments_collection)